In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.sparse import hstack
import pickle
from tqdm import tqdm
import pickle
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)


In [3]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Bidirectional, Embedding, Dropout, Input, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.feature_extraction.text import TfidfVectorizer
from keras import regularizers

In [5]:
cd ..

/home/harsh/projects/sembly-round2-harsh/task-1


In [6]:
cd ..

/home/harsh/projects/sembly-round2-harsh


In [7]:
df=pd.read_csv("task-1/data/cleaned_data.csv")
y_true=df['is_duplicate'][:40000]

In [8]:
model= Doc2Vec.load("task-1/data/doc2vec/d2v.model")
dv_=model.dv

In [9]:
question1_vectors=[]
question2_vectors=[]

In [10]:
for i in tqdm(range(40000)):
    question1_vectors.append(dv_[i])
    question2_vectors.append(dv_[i+40000])

100%|██████████| 40000/40000 [00:00<00:00, 135609.73it/s]


In [11]:
x=np.hstack((question1_vectors[:],question2_vectors[:]))

In [12]:
x_train,x_test,y_train,y_test = train_test_split(x,y_true,test_size=0.3,random_state=21)

In [13]:
x_train.shape

(28000, 400)

In [18]:
model=Sequential()
# model.add(Input(29422))
model.add(GRU(400,activation="relu",input_shape=(400,1,)))
# 
# model.add(Dropout(0.2))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
# model.add(Dense(64,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(32))
# model.add(Dense(32,activation="relu",kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation="sigmoid"))

In [19]:
callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_Accuracy',
            min_delta=1e-5,
            patience=10,
            verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath="task-1/saved_models/doc2vec/gru",
            monitor='val_loss', 
            mode='min', 
            save_best_only=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.2,
            patience=4, 
            min_lr=0.001)
    
]

# optimizer = tf.keras.optimizers.Adam(1e-5)
# loss = tf.keras.losses.BinaryCrossentropy()
# acc = tf.keras.metrics.Accuracy()

In [20]:
model.compile(loss = "binary_crossentropy", optimizer = "Adam", metrics = ["Accuracy"])

In [21]:
history = model.fit(x_train, y_train, epochs = 10, batch_size = 32, validation_data= (x_test, y_test), callbacks=callbacks, verbose=1)

Epoch 1/10
875/875 [==============================] - ETA: 0s - loss: 0.6516 - Accuracy: 0.6292
Epoch 1: val_loss improved from inf to 0.64366, saving model to task-1/saved_models/doc2vec/gru
INFO:tensorflow:Assets written to: task-1/saved_models/doc2vec/gru/assets
875/875 [==============================] - 442s 503ms/step - loss: 0.6516 - Accuracy: 0.6292 - val_loss: 0.6437 - val_Accuracy: 0.6240 - lr: 0.0010
Epoch 2/10
875/875 [==============================] - ETA: 0s - loss: 0.6410 - Accuracy: 0.6293
Epoch 2: val_loss improved from 0.64366 to 0.63846, saving model to task-1/saved_models/doc2vec/gru
INFO:tensorflow:Assets written to: task-1/saved_models/doc2vec/gru/assets
875/875 [==============================] - 464s 530ms/step - loss: 0.6410 - Accuracy: 0.6293 - val_loss: 0.6385 - val_Accuracy: 0.6240 - lr: 0.0010
Epoch 3/10
875/875 [==============================] - ETA: 0s - loss: 0.6391 - Accuracy: 0.6293
Epoch 3: val_loss did not improve from 0.63846
875/875 [================

KeyboardInterrupt: 